In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.append('../dependencies/')
import dataset_utils
import network_lrat_code as network
import utils
import zca

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.random.set_seed(42)

tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers

train_set = 'celeb_a'
norm = None
mode = 'color'
which_arch = ''
bg = 1

if mode == 'color':
    input_shape = (32, 32, 3)
    datasets = [
        'svhn_cropped',
        'cifar10',
        'celeb_a',
        'gtsrb',
        'compcars',
        'noise'
    ]
    num_filters = 64
    if bg:
        mutation_rate = 0.1
    else:
        mutation_rate = 0
elif mode == 'grayscale':
    input_shape = (32, 32, 1)
    datasets = [
    'mnist',
    'fashion_mnist',
    'emnist/letters',
    'sign_lang',
    'noise'
    ]
    num_filters = 32
    if bg:
        mutation_rate = 0.3
    else:
        mutation_rate = 0
    
if which_arch == 'lrat_arch':
    if bg:
        pre = 'lrat_arch_bg_reg/'
        reg_weight = 100
    else:
        pre = 'lrat_arch/'
        reg_weight = 0
    num_resnet = 5
    num_hierarchies = 1
    num_logistic_mix = 10
    num_filters = 160
    dropout_p = 0
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        0.0001,
        decay_steps=1,
        decay_rate=0.999995)
    epochs = 427
    use_weight_norm = False
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate, beta_1=0.95, beta_2=0.9995)
else:
    if bg:
        pre = 'lrat_bg_reg/'
        reg_weight = 0
    else:
        pre = 'lrat/'
        reg_weight = 0
    num_resnet = 2
    num_hierarchies = 4
    num_logistic_mix = 5
    num_filters = num_filters
    dropout_p = 0.3
    learning_rate = 1e-3
    use_weight_norm = True
    epochs = 100
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

if norm is None:
    dir_str = 'original'
elif norm == 'pctile-5':
    dir_str = 'pctile-5'
elif norm == 'channelwhiten':
    dir_str = 'zca'
elif norm == 'zca_original':
    dir_str = 'zca_original'
elif norm == 'histeq':
    dir_str = 'histeq'
    
model_dir = '../saved_models/' + pre + dir_str + '/' + train_set + '/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
if norm == 'zca_original':
    zca_transform = zca.compute_zca(train_set)
else:
    zca_transform = None
    
ds_train, ds_val, _ = dataset_utils.get_dataset(
      train_set,
      32,
      mode,
      normalize=norm,
      dequantize=False,
      visible_dist='categorical',
      zca_transform=zca_transform,
      mutation_rate=mutation_rate
  )

dist = network.PixelCNN(
      image_shape=input_shape,
      num_resnet=num_resnet,
      num_hierarchies=num_hierarchies,
      num_filters=num_filters,
      num_logistic_mix=num_logistic_mix,
      dropout_p=dropout_p,
      use_weight_norm=use_weight_norm,
      reg_weight=reg_weight
)

image_input = tfkl.Input(shape=input_shape)
log_prob = dist.log_prob(image_input)
model = tfk.Model(inputs=image_input, outputs=log_prob)
model.add_loss(-tf.reduce_mean(log_prob))
model.compile(optimizer=optimizer)

model.build([None] + list(input_shape))
# model.load_weights(model_dir+'weights')

# model.fit(
#         ds_train,
#         epochs=epochs,
#         validation_data=ds_val,
#         callbacks=[
#             utils.CNSModelCheckpoint(
#                 filepath=os.path.join(model_dir+'weights'),
#                 verbose=1, save_weights_only=True, save_best_only=True
#                 )
#         ],
#     )

model.load_weights(model_dir+'weights')

2022-05-09 18:09:34.492934: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-09 18:09:35.446306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7859 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1


Instructions for updating:
Use `tf.random.categorical` instead.


In [2]:
import tqdm
from scipy.io import savemat
from scipy.io import loadmat

if train_set == 'emnist/letters':
    train_set = 'emnist_letters'
    
probs = {}
if bg:
    if which_arch == 'lrat_arch':
        probs1 = loadmat('../probs/lrat_arch/' + dir_str + '/' + train_set + '.mat')
    else:
        probs1 = loadmat('../probs/lrat/' + dir_str + '/' + train_set + '.mat')

for dataset in datasets:
    
    _, _, ds_test = dataset_utils.get_dataset(
          dataset,
          256,
          mode,
          normalize=norm,
          dequantize=False,
          visible_dist='categorical',
          zca_transform=zca_transform,
          mutation_rate=0
      )
    tmp = []
    for test_batch in tqdm.tqdm(ds_test):
        tmp.append(dist.log_prob(tf.cast(test_batch, tf.float32),
                                        training=False).numpy())

    tmp = np.expand_dims(np.concatenate(tmp, axis=0),axis=-1)
    tmp = np.array(tmp)
    
    if bg:
        probs[dataset] = probs1[dataset] - tmp
    else:
        probs[dataset] = tmp

save_dir = '../probs/' + pre + dir_str + '/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

savemat(save_dir + train_set + '.mat', probs)

  0%|                                                                                                                                                                                    | 0/102 [00:00<?, ?it/s]2022-05-09 18:10:26.582641: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-09 18:10:28.067800: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [01:04<00:00,  1.62s/it]
156it [04:17,  1.65s/it]
50it [01:21,  1.63s/it]
53it [04:42,  5.33s/it]
40it [00:32,  1.25it/s]
